In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import re
import time
import plotly
import random
import tweepy
import telebot


In [2]:
class EksiBot():
    
    def __init__(self):
        os.environ["SELENIUM_SERVER_JAR"] = "selenium-server-standalone-2.41.0.jar"
        # note: I've put this jar file in the same folder as this python file

        self.driver = webdriver.Safari()

        # makes the browser wait if it can't find an element
        self.driver.implicitly_wait(5)
    
    def get_home_page(self):
        self.driver.get("https://eksisozluk1923.com")
        self.driver.implicitly_wait(5)
        
    def get_topics(self):
        
        topics_3 = self.driver.find_element(By.ID, "partial-index")
        self.topics_list = topics_3.text.split("\n\n\n ")
        self.topics_list_clean = []
        self.numbers = []
        for item in self.topics_list:
            if item == self.topics_list[2]:
                try:
                    number = re.search(r'\d+$', item.split("\n\n ")[1])
                    num = len(number.group(0))
                    self.numbers.append(num)
                    self.topics_list_clean.append(item.split("\n\n ")[1][:-(num+1)])
                except AttributeError:
                    continue

            else:
                try:
                    number = re.search(r'\d+$', item.split("\n\n")[0])
                    num = len(number.group(0))
                    self.numbers.append(num)
                    self.topics_list_clean.append(item.split("\n\n")[0][:-(num+1)])

                except AttributeError:
                    continue

                except:

                    number = re.search(r'\d+$', item)
                    num = len(number.group(0))
                    self.numbers.append(num)
                    self.topics_list_clean.append(item.split("\n\n")[0][:-(num+1)])
                    
        self.driver.implicitly_wait(5)
        return self.topics_list_clean            
    
    def get_topic(self, topic):
        
        
        self.driver.find_element(By.ID, "search-textbox").send_keys(topic + Keys.ENTER)
        #self.driver.find_element(By.CLASS_NAME, "last").click()
        time.sleep(1)
        
    def get_last(self):
        
        time.sleep(1)
        self.driver.find_element(By.CLASS_NAME, "last").click()
        
    def get_content(self):
        
        time.sleep(1)
        self.entries = self.driver.find_elements(By.CLASS_NAME, "content")
        
        self.entries_dict_clean = {}
        for i in range(len(self.entries)):
            entry = self.entries[i].text.split("\n   ")[1]
            self.entries_dict_clean[entry]=len(entry)


        return self.entries_dict_clean
    
    def iterate_over(self):
        
        self.all = {}
        topics = self.get_topics()
        for topic in topics:
            time.sleep(5)
            self.get_topic(topic)
            self.get_last(topic)
            content = self.get_content()
            self.all[topic]=content
            
        return self.all
        
        
        
    
    

In [3]:
driver = EksiBot()
driver.get_home_page()


In [4]:
topics = driver.get_topics()
result = '\n'.join([f'{index}: {string}' for index, string in enumerate(topics)])
topics


[" ankara bala'daki trafik kazasının görüntüleri",
 " gelecek partili 3 mv'nin ak parti'ye geçme kararı",
 " 9 eylül 2023 fenerbahçe'nin atatürk stadı kararı",
 " duygu özaslan'dan ilişki tavsiyesi",
 " a101'de satılan her şeyi yiyen çocuk kitabı",
 ' milli takıma teknik direktör tavsiyesi',
 ' seksin altın kuralı',
 ' 9 eylül 2023 sezgin tanrıkulu açıklaması',
 ' onca zorluğun arasında mağaraya resim yapan tip',
 ' eylülde tatile çıkan insan',
 ' klasik rus romanlarındaki 5 senedir giyilen palto',
 ' kemalizmin çağ dışı bir ideoloji olduğu gerçeği',
 ' arabasızlığın çok süper olması',
 ' beyne kazınmış biyoloji terimleri',
 ' emel sayın denince akla gelenler',
 ' menzilcilerin fantastik tövbe seansı',
 " kıvanç tatlıtuğ'un estetik yaptırması",
 ' dedeler tarafından kaçırılan yatırım fırsatları',
 ' cengiz ünder',
 ' ilişki durumunu bir doğa olayı ile anlatmak',
 " youtube'daki tamir videolarının işe yaramaması",
 " abdullah avcı'nın milli takımın başına geçmesi",
 ' açık seçik giyinmi

In [5]:
with open("new_topics.txt", mode="r+") as file:
    
    old_topics = file.readlines()

    for topic in topics:
        if not topic in old_topics:
            file.write(topic+"\n")
        else:
            continue


In [6]:
# Twitter API credentials
consumer_key = "" # api key
consumer_secret = "" # api key
access_token = ""
access_token_secret = ""
bearer_token = ""

# Authenticate with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create an API object
api = tweepy.API(auth)


In [7]:
client = tweepy.Client(bearer_token, consumer_key, consumer_secret, access_token, access_token_secret)

In [8]:
BOT_TOKEN=""

In [9]:
bot = telebot.TeleBot(BOT_TOKEN)
bot.send_message("@eksi_bott", result)


In [10]:
unposted = []
with open("tweets.txt", mode="a+") as file:
    
    for topic in topics:
        try:
            driver.get_topic(topic)
            driver.get_last()
            content = driver.get_content()
            postable = []
            for key, value in content.items():

                if len(topic+":\n\n"+key[1:]) <= 280:
                    postable.append(topic+":\n\n"+key[1:])
            print(len(postable))
            try:
                rand = random.randint(0, len(postable))
                time.sleep(5)
                client.create_tweet(text=postable[rand])
                file.write(postable[rand][1:])
                bot.send_message("@eksi_bott", postable[rand])

            except:
                unposted.append(topic)
                print(topic)
        except:
            unposted.append(topic)
            print(topic+"reklam")
        

2
5
2
3
5
 a101'de satılan her şeyi yiyen çocuk kitabı
10
 milli takıma teknik direktör tavsiyesi
 seksin altın kuralıreklam
 9 eylül 2023 sezgin tanrıkulu açıklamasıreklam
 onca zorluğun arasında mağaraya resim yapan tipreklam
 eylülde tatile çıkan insanreklam
 klasik rus romanlarındaki 5 senedir giyilen paltoreklam
 kemalizmin çağ dışı bir ideoloji olduğu gerçeğireklam
 arabasızlığın çok süper olmasıreklam
 beyne kazınmış biyoloji terimlerireklam
 emel sayın denince akla gelenlerreklam
 menzilcilerin fantastik tövbe seansıreklam
 kıvanç tatlıtuğ'un estetik yaptırmasıreklam
 dedeler tarafından kaçırılan yatırım fırsatlarıreklam
 cengiz ünderreklam
 ilişki durumunu bir doğa olayı ile anlatmakreklam
 youtube'daki tamir videolarının işe yaramamasıreklam
 abdullah avcı'nın milli takımın başına geçmesireklam
 açık seçik giyinmiş sevgiliyle utanmadan gezmekreklam
 izmir'in çok kötü kokmasıreklam
 risale-i nur okumanın huzur verdiği gerçeğireklam
 barış alper yılmazreklam
 getir'in değerinin

In [11]:
def share_singular_topic(topic):
    try:
        driver.get_topic(topic)
        driver.get_last()
        content = driver.get_content()
        postable = []
        for key, value in content.items():

            if len(topic+":\n\n"+key[1:]) <= 280:
                postable.append(topic+":\n\n"+key[1:])
        print(len(postable))
        try:
            rand = random.randint(0, len(postable))
            api.update_status(postable[rand])
            bot.send_message("@eksi_bott", postable[rand])

        except:
            print(topic)
    except:
        print(topic+"reklam")
        

In [12]:
#share_singular_topic("lise arkadaşlarıyla şimdiki ilişki durumu")


In [13]:
def share_selected_entry(topic):
    driver.get_topic(topic)
    driver.get_last()
    content = driver.get_content()
    postable = []
    for key, value in content.items():

        if len(topic+":\n\n"+key[1:]) <= 280:
            postable.append(topic+":\n\n"+key[1:])
    if len(postable) == 0:
        return
    print('\n'.join([f'{index}: {string}' for index, string in enumerate(postable)]))
    rand = int(input("Bir entry seçin: "))
    if rand == 13:
        return
    client.create_tweet(text=postable[rand])
    bot.send_message("@eksi_bott", postable[rand])


In [14]:
for topic in unposted:
    share_selected_entry(topic)

InvalidSessionIdException: Message: 
